In [1]:
import json
import re
import sentencepiece as spm

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [3]:
from prepro_utils import preprocess_text, encode_ids, encode_pieces

sp_model = spm.SentencePieceProcessor()
sp_model.Load('bert-base/sp10m.cased.v4.model')

with open('bert-base/sp10m.cased.v4.vocab') as fopen:
    v = fopen.read().split('\n')[:-1]
v = [i.split('\t') for i in v]
v = {i[0]: i[1] for i in v}

class Tokenizer:
    def __init__(self, v):
        self.vocab = v
        pass
    
    def tokenize(self, string):
        return encode_pieces(sp_model, string, return_unicode=False, sample=False)
    
    def convert_tokens_to_ids(self, tokens):
        return [sp_model.PieceToId(piece) for piece in tokens]
    
    def convert_ids_to_tokens(self, ids):
        return [sp_model.IdToPiece(i) for i in ids]
    
tokenizer = Tokenizer(v)

In [4]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling
import numpy as np
import json
import tensorflow as tf
import itertools
from unidecode import unidecode
import re

/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

/home/husein/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorboard/compat/ten

In [5]:
BERT_INIT_CHKPNT = 'bert-base/model.ckpt'
BERT_CONFIG = 'bert-base/bert_config.json'

In [6]:
import glob

files = glob.glob('../Malaya-Dataset/emotion/translated*')
files

['../Malaya-Dataset/emotion/translated-love',
 '../Malaya-Dataset/emotion/translated-happy',
 '../Malaya-Dataset/emotion/translated-fear',
 '../Malaya-Dataset/emotion/translated-sadness',
 '../Malaya-Dataset/emotion/translated-surprise',
 '../Malaya-Dataset/emotion/translated-anger']

In [7]:
files[0].split('translated-')[1]

'love'

In [8]:
import re
from unidecode import unidecode

def cleaning(string):
    string = unidecode(string)
    string = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', string)
    string = re.sub(r'[ ]+', ' ', string).strip().split()
    string = [w for w in string if w[0] != '@']
    return ' '.join(string)

In [9]:
texts, labels = [], []

for file in files:
    label = file.split('translated-')[1]
    with open(file) as fopen:
        t = fopen.read().split('\n')[:-1]
    t = [cleaning(s) for s in t]
    t = list(filter(None, t))
    texts.extend(t)
    print(label, len(t))
    labels.extend([label] * len(t))

love 15231
happy 19586
fear 19057
sadness 16052
surprise 9711
anger 18872


In [10]:
files = glob.glob('../Malaya-Dataset/emotion/*malaysia.json')
files

['../Malaya-Dataset/emotion/surprise-twitter-malaysia.json',
 '../Malaya-Dataset/emotion/happy-twitter-malaysia.json',
 '../Malaya-Dataset/emotion/love-twitter-malaysia.json',
 '../Malaya-Dataset/emotion/fear-twitter-malaysia.json',
 '../Malaya-Dataset/emotion/anger-twitter-malaysia.json',
 '../Malaya-Dataset/emotion/sadness-twitter-malaysia.json']

In [11]:
import json

for file in files:
    label = file.split('/')[-1].split('-')[0]
    with open(file) as fopen:
        t = json.load(fopen)
    t = [cleaning(s) for s in t]
    t = list(filter(None, t))
    texts.extend(t)
    print(label, len(t))
    labels.extend([label] * len(t))
    
len(texts), len(labels)

surprise 37778
happy 48924
love 59242
fear 18895
anger 51745
sadness 79233


(394326, 394326)

In [12]:
np.unique(labels, return_counts = True)

(array(['anger', 'fear', 'happy', 'love', 'sadness', 'surprise'],
       dtype='<U8'), array([70617, 37952, 68510, 74473, 95285, 47489]))

In [13]:
list(v.keys())[:10]

['<unk>',
 '<s>',
 '</s>',
 '<cls>',
 '<sep>',
 '<pad>',
 '<mask>',
 '<eod>',
 '<eop>',
 '.']

In [14]:
from tqdm import tqdm

input_ids = []

for text in tqdm(texts):
    tokens_a = tokenizer.tokenize(text)
    tokens = ["<cls>"] + tokens_a + ["<sep>"]
    input_id = tokenizer.convert_tokens_to_ids(tokens)
    
    input_ids.append(input_id)

100%|██████████| 394326/394326 [00:30<00:00, 13070.61it/s]


In [15]:
bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)

In [16]:
epoch = 10
batch_size = 60
warmup_proportion = 0.1
num_train_steps = int(len(texts) / batch_size * epoch)
num_warmup_steps = int(num_train_steps * warmup_proportion)

In [17]:
bert_config.hidden_size

768

In [18]:
from sklearn.preprocessing import LabelEncoder

labels = LabelEncoder().fit_transform(labels)
labels = labels.tolist()

In [19]:
def create_initializer(initializer_range=0.02):
    return tf.truncated_normal_initializer(stddev=initializer_range)

class Model:
    def __init__(
        self,
        dimension_output,
        learning_rate = 2e-5,
        training = True,
    ):
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None])
        
        model = modeling.BertModel(
            config=bert_config,
            is_training=training,
            input_ids=self.X,
            use_one_hot_embeddings=False)
        
        output_layer = model.get_sequence_output()
        output_layer = tf.layers.dense(
            output_layer,
            bert_config.hidden_size,
            activation=tf.tanh,
            kernel_initializer=create_initializer())
        self.logits_seq = tf.layers.dense(output_layer, dimension_output,
                                         kernel_initializer=create_initializer())
        self.logits_seq = tf.identity(self.logits_seq, name = 'logits_seq')
        self.logits = self.logits_seq[:, 0]
        self.logits = tf.identity(self.logits, name = 'logits')
        
        self.cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits = self.logits, labels = self.Y
            )
        )
        
        self.optimizer = optimization.create_optimizer(self.cost, learning_rate, 
                                                       num_train_steps, num_warmup_steps, False)
        correct_pred = tf.equal(
            tf.argmax(self.logits, 1, output_type = tf.int32), self.Y
        )
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [20]:
dimension_output = 6
learning_rate = 2e-5

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
    learning_rate
)

sess.run(tf.global_variables_initializer())
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'bert')
saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, BERT_INIT_CHKPNT)




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from bert-base/model.ckpt


In [21]:
from sklearn.model_selection import train_test_split

train_input_ids, test_input_ids, train_Y, test_Y = train_test_split(
    input_ids, labels, test_size = 0.2
)

In [22]:
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences

In [23]:
from tqdm import tqdm
import time

EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 2, 0, 0, 0

while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n' % (EPOCH))
        break

    train_acc, train_loss, test_acc, test_loss = [], [], [], []
    pbar = tqdm(
        range(0, len(train_input_ids), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        index = min(i + batch_size, len(train_input_ids))
        batch_x = train_input_ids[i: index]
        batch_x = pad_sequences(batch_x, padding='post')
        batch_y = train_Y[i: index]
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
            },
        )
        train_loss.append(cost)
        train_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    pbar = tqdm(range(0, len(test_input_ids), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_input_ids))
        batch_x = test_input_ids[i: index]
        batch_x = pad_sequences(batch_x, padding='post')
        batch_y = test_Y[i: index]
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
            },
        )
        test_loss.append(cost)
        test_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    train_loss = np.mean(train_loss)
    train_acc = np.mean(train_acc)
    test_loss = np.mean(test_loss)
    test_acc = np.mean(test_acc)
        
    if test_acc > CURRENT_ACC:
        print(
            'epoch: %d, pass acc: %f, current acc: %f'
            % (EPOCH, CURRENT_ACC, test_acc)
        )
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )
    EPOCH += 1

train minibatch loop:   0%|          | 1/5258 [00:00<16:20,  5.36it/s, accuracy=0.833, cost=0.265]

epoch: 0, pass acc: 0.000000, current acc: 0.864821
time taken: 1257.505273103714
epoch: 0, training loss: 0.443955, training acc: 0.810663, valid loss: 0.271198, valid acc: 0.864821



train minibatch loop:   0%|          | 1/5258 [00:00<16:24,  5.34it/s, accuracy=0.85, cost=0.202]

epoch: 1, pass acc: 0.864821, current acc: 0.871441
time taken: 1252.8684689998627
epoch: 1, training loss: 0.259072, training acc: 0.868134, valid loss: 0.248496, valid acc: 0.871441



train minibatch loop:   0%|          | 1/5258 [00:00<16:45,  5.23it/s, accuracy=0.85, cost=0.188]

...

epoch: 3, pass acc: 0.871441, current acc: 0.872670
time taken: 1252.0703406333923
epoch: 3, training loss: 0.211508, training acc: 0.882059, valid loss: 0.251088, valid acc: 0.872670



train minibatch loop:   0%|          | 1/5258 [00:00<16:15,  5.39it/s, accuracy=0.867, cost=0.162]

time taken: 1261.1393103599548
epoch: 4, training loss: 0.197253, training acc: 0.887104, valid loss: 0.272873, valid acc: 0.870588



test minibatch loop: 100%|██████████| 1315/1315 [01:28<00:00, 14.81it/s, accuracy=0.885, cost=0.176] 

time taken: 1253.5724835395813
epoch: 5, training loss: 0.185673, training acc: 0.890784, valid loss: 0.281502, valid acc: 0.868353

break epoch:6



In [24]:
real_Y, predict_Y = [], []

pbar = tqdm(
    range(0, len(test_input_ids), batch_size), desc = 'validation minibatch loop'
)
for i in pbar:
    index = min(i + batch_size, len(test_input_ids))
    batch_x = test_input_ids[i: index]
    batch_x = pad_sequences(batch_x, padding='post')
    batch_y = test_Y[i: index]
    
    predict_Y += np.argmax(sess.run(model.logits,
            feed_dict = {
            model.X: batch_x,
            },
    ), 1, ).tolist()
    real_Y += batch_y

validation minibatch loop: 100%|██████████| 1315/1315 [01:26<00:00, 15.13it/s]


In [25]:
from sklearn import metrics

print(
    metrics.classification_report(
        real_Y, predict_Y, target_names = ['anger', 'fear', 'happy', 'love', 'sadness', 'surprise'],
        digits = 5
    )
)

              precision    recall  f1-score   support

       anger    0.92201   0.94056   0.93119     14316
        fear    0.89599   0.84937   0.87205      7515
       happy    0.92960   0.91353   0.92149     13832
        love    0.94360   0.94205   0.94282     14686
     sadness    0.78613   0.85664   0.81987     19043
    surprise    0.73266   0.62888   0.67681      9474

    accuracy                        0.86970     78866
   macro avg    0.86833   0.85517   0.86071     78866
weighted avg    0.86932   0.86970   0.86858     78866

